# Create decision boundary 

We use certainty ratios to extract data samples that lie on a decision boundary. 

More information about this play can be found in the Spotlight documentation: [Detect decision boundaries](https://renumics.com/docs/playbook/decision-boundary)

For more data-centric AI workflows, check out our [Awesome Open Data-centric AI](https://github.com/Renumics/awesome-open-data-centric-ai) list on Github.

## tldr

In [ ]:
#@title Install required packages with PIP

!pip install renumics-spotlight datasets

In [ ]:
#@title Play as copy-n-paste functions

import datasets
import numpy as np
import pandas as pd
from renumics import spotlight
import requests
import json

def boundary_score(df, probabilities_name='probabilities'):
    def compute_score(probs):      
        indices=np.argsort(probs)[::-1]   
        score = [indices[0], indices[1], probs[indices[0]],probs[indices[1]] ] 
     
        return score
    
    df_out=pd.DataFrame()
    temp_scores=[compute_score(x) for x in df[probabilities_name]] 
    df_out['decision_boundary_score']=[x[3]/x[2] for x in temp_scores]
    df_out['decision_boundary_alternate_label']=[x[1] for x in temp_scores]
    
       
    return df_out



## Step-by-step example on CIFAR-100

### Load CIFAR-100 from Huggingface hub and convert it to Pandas dataframe

In [ ]:
dataset = datasets.load_dataset("renumics/cifar100-enriched", split="all")

df = dataset.to_pandas()

### Compute decision boundary score and alternate label

In [ ]:
df_boundary = boundary_score(df)
df = pd.concat([df, df_boundary], axis=1)
df['decision_boundary_alternate_label_str']=[dataset.features["fine_label"].int2str(x) for x in df['decision_boundary_alternate_label']]

### Inspect decision boundary with Spotlight

> ⚠️ Running Spotlight in Colab currently has severe limitations (slow, no similarity map, no layouts) due to Colab restrictions (e.g. no websocket support). Run the notebook locally for the full Spotlight experience

In [ ]:
df_show = df.drop(columns=['embedding', 'probabilities'])


# handle google colab differently
import sys
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    #visualization in Google Colab only works in chrome and does not support websockets, we need some hacks to visualize something
    df_show=df_show[:10000]
    df_show['embx'] =  [emb[0] for emb in df_show['embedding_reduced'] ]
    df_show['emby'] =  [emb[1] for emb in df_show['embedding_reduced'] ]
    port=50123
    spotlight.show(df_show, port=port, dtype={"image": spotlight.Image, "embedding_reduced": spotlight.Embedding})  
    from google.colab.output import eval_js  # type: ignore
    print(str(eval_js(f"google.colab.kernel.proxyPort({port}, {{'cache': true}})")))

else:
    df_show = df.drop(columns=['embedding', 'probabilities'])
    layout_url = "https://raw.githubusercontent.com/Renumics/spotlight/playbook_initial_draft/playbook/rookie/decision_boundary_layout.json"
    response = requests.get(layout_url)
    layout = spotlight.layout.nodes.Layout(**json.loads(response.text))
    spotlight.show(df_show, dtype={"image": spotlight.Image, "embedding_reduced": spotlight.Embedding}, layout=layout)